In [1]:
from transformers import BertTokenizer, BertForMaskedLM, BertTokenizer

model_path = tokenizer_path = "bert-base-uncased"  
model = BertForMaskedLM.from_pretrained(model_path, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [2]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /home/sha43/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import random
def tokenize_function(examples):
    for i in range(len(examples["text"])):
        texttt = examples["text"][i].split()
        random.shuffle(texttt)
        examples["text"][i] = " ".join(texttt)

    return tokenizer(examples["text"], truncation=True)

In [4]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=1, remove_columns=["text"])

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [5]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [6]:
# block_size = tokenizer.model_max_length
block_size = 128

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/10 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    f"finetuned-scrumbled-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=6
)

In [8]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [10]:
trainer.train()

/home/sha43/anaconda3/envs/diploma/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18535
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 13902
  Number of trainable parameters = 109514298


Epoch,Training Loss,Validation Loss
1,5.108700,5.069732
2,4.978300,5.054367
3,4.934700,5.041335
4,4.906500,4.962860
5,4.871800,4.976539
6,4.803700,4.975343


Saving model checkpoint to finetuned-scrumbled-wikitext2/checkpoint-500
Configuration saved in finetuned-scrumbled-wikitext2/checkpoint-500/config.json
Model weights saved in finetuned-scrumbled-wikitext2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to finetuned-scrumbled-wikitext2/checkpoint-1000
Configuration saved in finetuned-scrumbled-wikitext2/checkpoint-1000/config.json
Model weights saved in finetuned-scrumbled-wikitext2/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to finetuned-scrumbled-wikitext2/checkpoint-1500
Configuration saved in finetuned-scrumbled-wikitext2/checkpoint-1500/config.json
Model weights saved in finetuned-scrumbled-wikitext2/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to finetuned-scrumbled-wikitext2/checkpoint-2000
Configuration saved in finetuned-scrumbled-wikitext2/checkpoint-2000/config.json
Model weights saved in finetuned-scrumbled-wikitext2/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num ex

TrainOutput(global_step=13902, training_loss=4.955346790325246, metrics={'train_runtime': 1779.3212, 'train_samples_per_second': 62.501, 'train_steps_per_second': 7.813, 'total_flos': 7317751807872000.0, 'train_loss': 4.955346790325246, 'epoch': 6.0})